In [878]:
import sys
import os
from subprocess import Popen, PIPE
import re
import numpy as np

In [723]:
PRAAT= '/usr/bin/praat'
#PRAAT = 'D:\Downloads\praat6104_win64\Praat.exe'

diphone_folder = "/home/aletg/ph-tp1/difonos/"
#diphone_folder = "D:\Documents\Facu\ph-tp1\difonos\\"

In [724]:
def run_praat(script_file, args):
    args= [PRAAT, '--run', script_file, args]
    p = Popen(args, stdout=PIPE)
    p.wait()

Lista de archivos de audio - concatenar audio y escribir a archivo

In [1124]:
def concatenate(filenames, output_name):
    script = ""
    for i, f in enumerate(filenames):
        script += "Read from file: \"" + f + "\"\n"
        script += "selectObject: \"Sound " + f[-6:-4] + "\"\n"
        script += "Rename: \"" + f[-6:-4] + str(i) + "\"\n"

    script += "selectObject: \"Sound " + filenames[0][-6:-4] + str(0) + "\"\n"
    for i, f in enumerate(filenames[1:]):
        script += "plusObject: \"Sound " + f[-6:-4] + str(i+1) + "\"\n"

    script += "Concatenate recoverably\n"
    script += "selectObject: \"Sound chain\"\n"
    script += "Write to WAV file: \"" + output_name + "\"\n"
    script += "Save as text file: \"" + output_name + ".TextGrid\"\n"

    with open("concatenar.praat", "w") as cf:
        cf.write(script)

    run_praat("concatenar.praat", "")
    #os.remove("concatenar.praat")

Obtener lista de difonos a partir de una palabra

In [1091]:
all_diphones = ["-e", "-E", "-f", "-k", "-m", "-s", "-t", "e-", "E-",
           "s-", "ee", "eE", "Ee", "EE", "fe", "fE", "ke", "kE", 
           "me", "mE", "se", "sE", "te", "tE", "re", "rE", "ef",
           "Ef", "ek", "Ek", "em", "Em", "es", "Es", "et", "Et",
           "er", "Er", "fr", "kr", "tr", "sf", "sk", "st", "ss",
           "sm"]

wav = ".wav"

def diphone_list(word):
    diphones = []
    word  = "-" + word + "-"
    for i, _ in enumerate(word[:-1]):
        diphone = word[i:i+2]
        if not diphone in all_diphones:
            print("ERROR: invalid word. Try again.")
            sys.exit(1)
        diphones.append(diphone)
    print(diphones)
    # agregar prefijo y sufijo
    return diphones

#list(map(lambda s: diphone_folder + s + wav, diphone_list))

def diphone_to_path(diphone):
    return diphone_folder + diphone + wav

In [1045]:
pitch_start_range = 90
pitch_end_range = 300

def extract_pitch_track(filename):
    args = " "
    args += filename + ".wav " + filename + ".PitchTier " + str(pitch_start_range) + " " + str(pitch_end_range)
    run_praat("extraer-pitch-track.praat", args )

def replace_pitch_track(inputSound, inputPitch, outSound):
    args = " "
    args += inputSound + " " + inputPitch + " " + outSound + " " + str(pitch_start_range) + " " + str(pitch_end_range)
    run_praat("reemplazar-pitch-track.praat", args )

def modify_pitch_track( pitchTierIn, pitchTierOut, f=lambda p, t, pmin, pmax: 2*p ):
    with open(pitchTierIn, 'r') as inF:
        file = inF.read()

    file = file.split("\n")
    pmin = 300.0
    pmax = 0.0
    xmin = 5.0
    xmax = -1.9
    for line in file:
        if "number" in line:
            number = float(line[12:])
            if number < xmin:
                xmin = number
            if number > xmax:
                xmax = number
        elif "value" in line:
            pitch = float(line[12:])
            if pitch < pmin:
                pmin = pitch
            if pitch > pmax:
                pmax = pitch

    with open(pitchTierOut, "w") as outF:
        number = 0.0
        for line in file:
            if "number" in line:
                number = float(line[12:])
            elif "value" in line:
                pitch = float(line[12:])
                pitch = f(pitch, (number-xmin)/(xmax - xmin), pmin, pmax)
                line = "    value = " + str(pitch)
            outF.write(line + "\n")

In [1005]:
modify_pitch_track( "./test/estemeketrefe.PitchTier", "./test/estemeketrefe_agudo.PitchTier", lambda x, t, min, max: np.log(t+1)*x )

0.0920104166666666
1.5836770833333333


In [1188]:
aux_folder = "./aux/"
def tts(word, output_filename, pregunta):
    diphones = diphone_list(word)
    if not pregunta:
        concatenate([diphone_to_path(d) for d in diphones], output_filename)
    else:
        i = 0
        j = 0
        output = []
        while i < len(diphones):
            if diphones[:-1] is not "s-" and i == len(diphones)-2 :
                new_aux = folder_aux + "a" + str(j)
                acum = [diphone_to_path(diphones[i]), diphone_to_path(diphones[i+1])]
                mergear_y_variar(acum, new_aux, f_ascendente_300)
                output.append(new_aux + wav)
                break
            elif diphones[:-1] is "s-" and i == len(diphones)-3:
                new_aux = folder_aux + "a" + str(j)
                acum = [diphone_to_path(diphones[i]), diphone_to_path(diphones[i+1])]
                mergear_y_variar(acum, new_aux, f_ascendente_300)
                output.append(new_aux + wav)
                output.append(diphone_to_path("s-"))
                break
            elif 'E' in diphones[i]:
                new_aux = folder_aux + "a" + str(j)
                acum = [diphone_to_path(diphones[i]), diphone_to_path(diphones[i+1])]
                #if i+2 < len(diphones):
                #    acum.append(diphone_to_path(diphones[i+2]))
                #    i += 1
                i += 1
                mergear_y_variar(acum, new_aux, f_ascendente_300)
                output.append(new_aux + wav)
                j += 1
            else:
                output.append(diphone_to_path(diphones[i]))
            i += 1
        concatenate(output, output_filename)

In [1193]:
tts("EsemeketrEfe", "./prueba.wav", True)

['-E', 'Es', 'se', 'em', 'me', 'ek', 'ke', 'et', 'tr', 'rE', 'Ef', 'fe', 'e-']
['/home/aletg/ph-tp1/difonos/-E.wav', '/home/aletg/ph-tp1/difonos/Es.wav']
 ./aux/au.wav ./aux/au.PitchTier 90 300
['/home/aletg/ph-tp1/difonos/rE.wav', '/home/aletg/ph-tp1/difonos/Ef.wav']
 ./aux/au.wav ./aux/au.PitchTier 90 300
['/home/aletg/ph-tp1/difonos/fe.wav', '/home/aletg/ph-tp1/difonos/e-.wav']
 ./aux/au.wav ./aux/au.PitchTier 90 300


In [1192]:
tts("freskEteses", "./prueba.wav", True)

['-f', 'fr', 're', 'es', 'sk', 'kE', 'Et', 'te', 'es', 'se', 'es', 's-']
['/home/aletg/ph-tp1/difonos/kE.wav', '/home/aletg/ph-tp1/difonos/Et.wav']
 ./aux/au.wav ./aux/au.PitchTier 90 300
['/home/aletg/ph-tp1/difonos/es.wav', '/home/aletg/ph-tp1/difonos/s-.wav']
 ./aux/au.wav ./aux/au.PitchTier 90 300


In [1179]:
def variar_pitch(inF, outF, f):
    extract_pitch_track( inF )
    modify_pitch_track( inF + ".PitchTier", outF + ".PitchTier", f )
    replace_pitch_track( inF + wav, outF + ".PitchTier", outF + wav )

def mergear_y_variar(lista_difonos, output, f):
    aux = aux_folder + "au"
    concatenate(lista_difonos, aux + wav)
    print(lista_difonos)
    variar_pitch(aux, output, f)
    os.remove(aux + wav)
    os.remove(aux + wav + ".TextGrid")
    os.remove(aux + ".PitchTier")

def f_ascendente_250(p, t, pmin, pmax):
    maximo = 250
    res = (np.log((2**(maximo/pmax)-2)*t + 2)/np.log(2))*p
    return res

def f_ascendente_300(p, t, pmin, pmax):
    maximo = 350
    res = (np.log((2**(maximo/pmax)-2)*t + 2)/np.log(2))*p
    return res

def f_ascendente250_300(p, t, pmin, pmax):
    maximo = 300
    minimo = 250
    res = (np.log((2**(maximo/pmax)-2**(minimo/pmin))*t + 2**(minimo/pmin))/np.log(2))*p
    return res

def f_ascendente_300(p, t, pmin, pmax):
    maximo = 300
    res = (maximo-pmin)*t + pmin
    #res = (np.log((2**(maximo)-2**(pmin))*t + 2**(pmin))/np.log(2))
    return res

In [1127]:
def test(word):
    tts(word, "./test/" + word + ".wav", False)
    
def test_dbg(list, name):
    output_audio(list, "./test/" + name + ".wav")    

In [1096]:
test("estemeketrefe")
variar_pitch("./test/estemeketrefe", "./test/estemeketrefe_agudo", f_ascendente250_300) #lambda x, t: np.log(t+1)*x)

['-e', 'es', 'st', 'te', 'em', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']
 ./test/estemeketrefe.wav ./test/estemeketrefe.PitchTier 90 300


In [1046]:
variar_pitch("./difonos_/-e", "./difonos_/a", f_ascendente_250) #lambda x, t: np.log(t+1)*x)

variar_pitch("./difonos_/es", "./difonos_/as", f_ascendente250_300) #lambda x, t: np.log(t+1)*x)

output_audio(["./difonos_/-e.wav", "./difonos_/es.wav"], "./difonos_/as.wav")
variar_pitch("./difonos_/as", "./difonos_/os", f_ascendente_300)

 ./difonos_/-e.wav ./difonos_/-e.PitchTier 90 300
 ./difonos_/es.wav ./difonos_/es.PitchTier 90 300


 ./difonos_/as.wav ./difonos_/as.PitchTier 90 300


In [1103]:
output_audio(["./difonos_/os.wav", "./difonos/se.wav", "./difonos/e-.wav"], "./a.wav")

In [1110]:
mergear_y_variar(["./difonos/sE.wav","./difonos/Es.wav"], "ou", f_ascendente_300 )


 ./aux/au.wav ./aux/au.PitchTier 90 300


Ejemplo de ejecución del TTS

In [1092]:
test("fefefEfe")

['-f', 'fe', 'ef', 'fe', 'ef', 'fE', 'Ef', 'fe', 'e-']


In [143]:
test("fefefe")

['-f', 'fe', 'ef', 'fe', 'ef', 'fe', 'e-']
['D:\\Downloads\\praat6104_win64\\Praat.exe', '--run', 'concatenar.praat', '']


In [87]:
test("sefefe")

['-s', 'se', 'ef', 'fe', 'ef', 'fe', 'e-']


In [88]:
test("sefEfefese")

['-s', 'se', 'ef', 'fE', 'Ef', 'fe', 'ef', 'fe', 'es', 'se', 'e-']


In [78]:
test("sefefes")

['-s', 'se', 'ef', 'fe', 'ef', 'fe', 'es', 's-']


In [79]:
test("efefe")

['-e', 'ef', 'fe', 'ef', 'fe', 'e-']


In [89]:
test("mes")
test_dbg(diphone_list("mes")[1:], "mesa")

['-m', 'me', 'es', 's-']
['-m', 'me', 'es', 's-']


In [60]:
test("es")

['-e', 'es', 's-']


In [61]:
test("mefes")

['-m', 'me', 'ef', 'fe', 'es', 's-']


In [62]:
test("mese")

['-m', 'me', 'es', 'se', 'e-']


In [90]:
test("meme")
test_dbg(diphone_list("eme")[1:], "eme!")

['-m', 'me', 'em', 'me', 'e-']
['-e', 'em', 'me', 'e-']


In [91]:
test("emes")

['-e', 'em', 'me', 'es', 's-']


In [65]:
test("eme")

['-e', 'em', 'me', 'e-']


In [66]:
test("ke")

['-k', 'ke', 'e-']


In [67]:
test("keke")

['-k', 'ke', 'ek', 'ke', 'e-']


In [92]:
test("ekefeke")

['-e', 'ek', 'ke', 'ef', 'fe', 'ek', 'ke', 'e-']


In [93]:
test("esefeseke")

['-e', 'es', 'se', 'ef', 'fe', 'es', 'se', 'ek', 'ke', 'e-']


In [546]:
test("kese")

['-k', 'ke', 'es', 'se', 'e-']


In [547]:
test("keses")

['-k', 'ke', 'es', 'se', 'es', 's-']


In [94]:
test("kekeke")

['-k', 'ke', 'ek', 'ke', 'ek', 'ke', 'e-']


In [549]:
test("te")

['-t', 'te', 'e-']


In [550]:
test("tese")

['-t', 'te', 'es', 'se', 'e-']


In [551]:
test("teketeke")

['-t', 'te', 'ek', 'ke', 'et', 'te', 'ek', 'ke', 'e-']


In [552]:
test("tetete")

['-t', 'te', 'et', 'te', 'et', 'te', 'e-']


In [553]:
test("fetre")

['-f', 'fe', 'et', 'tr', 're', 'e-']


In [95]:
test("meketrefe")
test("esemeketrefe")

['-m', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']
['-e', 'es', 'se', 'em', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']


In [555]:
test("ererere")
test("eskereskere")

['-e', 'er', 're', 'er', 're', 'er', 're', 'e-']
['-e', 'es', 'sk', 'ke', 'er', 're', 'es', 'sk', 'ke', 'er', 're', 'e-']


In [556]:
test("este")

['-e', 'es', 'st', 'te', 'e-']


In [139]:
test("estemeketrefe")
test("estemekestefe")

['-e', 'es', 'st', 'te', 'em', 'me', 'ek', 'ke', 'et', 'tr', 're', 'ef', 'fe', 'e-']
['D:\\Downloads\\praat6104_win64\\Praat.exe', '--run', 'concatenar.praat', '']
['-e', 'es', 'st', 'te', 'em', 'me', 'ek', 'ke', 'es', 'st', 'te', 'ef', 'fe', 'e-']
['D:\\Downloads\\praat6104_win64\\Praat.exe', '--run', 'concatenar.praat', '']


In [558]:
test("efre")
test("fre")

['-e', 'ef', 'fr', 're', 'e-']
['-f', 'fr', 're', 'e-']


In [559]:
test("fretre")

['-f', 'fr', 're', 'et', 'tr', 're', 'e-']


In [96]:
test("freskete")
test("kefreskete")
test("kekefreskete")

['-f', 'fr', 're', 'es', 'sk', 'ke', 'et', 'te', 'e-']
['-k', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'ke', 'et', 'te', 'e-']
['-k', 'ke', 'ek', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'ke', 'et', 'te', 'e-']


In [561]:
test("kekEke")

['-k', 'ke', 'ek', 'kE', 'Ek', 'ke', 'e-']


In [97]:
test("kekefreskEke")

['-k', 'ke', 'ek', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'kE', 'Ek', 'ke', 'e-']


In [563]:
test("kefreskEte")
test("kEfreskEte")

['-k', 'ke', 'ef', 'fr', 're', 'es', 'sk', 'kE', 'Et', 'te', 'e-']
['-k', 'kE', 'Ef', 'fr', 're', 'es', 'sk', 'kE', 'Et', 'te', 'e-']


In [564]:
test("ekre")
test("tekre")
test("kekrete")

['-e', 'ek', 'kr', 're', 'e-']
['-t', 'te', 'ek', 'kr', 're', 'e-']
['-k', 'ke', 'ek', 'kr', 're', 'et', 'te', 'e-']


In [98]:
test("esfe")
test("esfesfesfe")
test("esfesfesfesfesfesfesfe")

['-e', 'es', 'sf', 'fe', 'e-']
['-e', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'e-']
['-e', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'es', 'sf', 'fe', 'e-']


In [566]:
test_dbg(diphone_list("esfes")[1:], "esfe!")

['-e', 'es', 'sf', 'fe', 'es', 's-']


In [567]:
test("este")
test("esste")

['-e', 'es', 'st', 'te', 'e-']
['-e', 'es', 'ss', 'st', 'te', 'e-']


In [568]:
test_dbg(diphone_list("esme")[2:], "esme!")
test("esme")
test("esmesmesme")

['-e', 'es', 'sm', 'me', 'e-']
['-e', 'es', 'sm', 'me', 'e-']
['-e', 'es', 'sm', 'me', 'es', 'sm', 'me', 'es', 'sm', 'me', 'e-']


In [569]:
test("Efe")

['-E', 'Ef', 'fe', 'e-']


In [570]:
test("E")
test("EfE")
test("EfefefEfefE")

['-E', 'E-']
['-E', 'Ef', 'fE', 'E-']
['-E', 'Ef', 'fe', 'ef', 'fe', 'ef', 'fE', 'Ef', 'fe', 'ef', 'fE', 'E-']


In [571]:
test("eEke")

['-e', 'eE', 'Ek', 'ke', 'e-']


In [572]:
test("Eeke")
test("Eefe")

['-E', 'Ee', 'ek', 'ke', 'e-']
['-E', 'Ee', 'ef', 'fe', 'e-']


In [573]:
test("efe")

['-e', 'ef', 'fe', 'e-']


In [99]:
test("EEEEEEEeeeEE")
test("eeeeeeeee")

['-E', 'EE', 'EE', 'EE', 'EE', 'EE', 'EE', 'Ee', 'ee', 'ee', 'eE', 'EE', 'E-']
['-e', 'ee', 'ee', 'ee', 'ee', 'ee', 'ee', 'ee', 'ee', 'e-']


In [575]:
test("fEEfEE")

['-f', 'fE', 'EE', 'Ef', 'fE', 'EE', 'E-']


In [576]:
test("mEfe")
test("mEfemE")

['-m', 'mE', 'Ef', 'fe', 'e-']
['-m', 'mE', 'Ef', 'fe', 'em', 'mE', 'E-']


In [577]:
test("sE")
test("sEfE")
test("sEke")

['-s', 'sE', 'E-']
['-s', 'sE', 'Ef', 'fE', 'E-']
['-s', 'sE', 'Ek', 'ke', 'e-']


In [578]:
test("tE")
test("tEfE")
test("tEkE")
test("etEkE")

['-t', 'tE', 'E-']
['-t', 'tE', 'Ef', 'fE', 'E-']
['-t', 'tE', 'Ek', 'kE', 'E-']
['-e', 'et', 'tE', 'Ek', 'kE', 'E-']


In [579]:
test("erE")
test("erEfE")
test("erEkE")
test("erEkE")

['-e', 'er', 'rE', 'E-']
['-e', 'er', 'rE', 'Ef', 'fE', 'E-']
['-e', 'er', 'rE', 'Ek', 'kE', 'E-']
['-e', 'er', 'rE', 'Ek', 'kE', 'E-']


In [580]:
test("erEme")
test("erEmE")
test("ereme")
test("erEmefe")

['-e', 'er', 'rE', 'Em', 'me', 'e-']
['-e', 'er', 'rE', 'Em', 'mE', 'E-']
['-e', 'er', 're', 'em', 'me', 'e-']
['-e', 'er', 'rE', 'Em', 'me', 'ef', 'fe', 'e-']


In [581]:
test("erEse")
test("erEsE")
test("erese")
test("erEsefe")

['-e', 'er', 'rE', 'Es', 'se', 'e-']
['-e', 'er', 'rE', 'Es', 'sE', 'E-']
['-e', 'er', 're', 'es', 'se', 'e-']
['-e', 'er', 'rE', 'Es', 'se', 'ef', 'fe', 'e-']


In [100]:
test("erEre")
test("erErE")
test("erere")
test("erErefe")

['-e', 'er', 'rE', 'Er', 're', 'e-']
['-e', 'er', 'rE', 'Er', 'rE', 'E-']
['-e', 'er', 're', 'er', 're', 'e-']
['-e', 'er', 'rE', 'Er', 're', 'ef', 'fe', 'e-']


In [862]:
test("skereskere")

['-s', 'sk', 'ke', 'er', 're', 'es', 'sk', 'ke', 'er', 're', 'e-']


In [866]:
test("ererE")
test("eresE")
test("erekE")
test("eretE")

['-e', 'er', 're', 'er', 'rE', 'E-']
['-e', 'er', 're', 'es', 'sE', 'E-']
['-e', 'er', 're', 'ek', 'kE', 'E-']
['-e', 'er', 're', 'et', 'tE', 'E-']


In [867]:
test("kEfreskEtekeEsteestE")

['-k', 'kE', 'Ef', 'fr', 're', 'es', 'sk', 'kE', 'Et', 'te', 'ek', 'ke', 'eE', 'Es', 'st', 'te', 'ee', 'es', 'st', 'tE', 'E-']


In [868]:
test("kEmemEfekeEsteestE")

['-k', 'kE', 'Em', 'me', 'em', 'mE', 'Ef', 'fe', 'ek', 'ke', 'eE', 'Es', 'st', 'te', 'ee', 'es', 'st', 'tE', 'E-']


In [ ]:
def leer_pitch_track(filename):
    with open(filename, 'r') as f:
        res = f.read()
    return res